#BIO Sensor 데이터 초기 전처리

각 Sesstion에서 script별로 나눠져있는 바이오 데이터 합치기

In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

# 불러올 파일 경로
path_dir = './dataset/KEMDy20_v1_1/EDA'
# 지정된 경로의 파일 모두 리스트에 불러오기
sess_FList = os.listdir(path_dir)

# 저장할 파일 경로
drop_dir ='./dataset/KEMDy20_v1_1/sensor/EDA/'

# 세션에서 스크립트별로 나눠져있던 eda 데이터 합치기기
for folder in sess_FList:
  lst = []
  sess_save=[] # ('session_number', 'EDA')
  eda_FList = os.listdir(path_dir+'/'+folder)
  for script in eda_FList:
      with open('./dataset/KEMDy20_v1_1/EDA/'+folder+'/'+script) as lines:
        b = list(map(str, script.split('_')))
        for line in lines:
          data = line.strip().split(',')
          if len(data)==3:
            a = list(map(str, data[2].split('_')))
            data[2] = data[2].replace(a[0],b[0])
            lst.append(data)
  df = pd.DataFrame(lst, columns=['data','date','sess'])
  df = df.astype({'data':float})  
  
  sess_name = df.sess.unique()
  for sess in tqdm(sess_name):
    eda=list(df[df.sess == sess].data.values)
    sess_list = [sess, eda]
    sess_save.append(sess_list)
  sess_save_df = pd.DataFrame(sess_save, columns=['session', 'eda'])
  sess_save_df.to_pickle(drop_dir+folder+'_eda.pkl')

In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

# 불러올 파일 경로
path_dir = './dataset/KEMDy20_v1_1/TEMP'
# 지정된 경로의 파일 모두 리스트에 불러오기
sess_FList = os.listdir(path_dir)

# 저장할 파일 경로
drop_dir ='./dataset/KEMDy20_v1_1/sensor/Temp/'

# 세션에서 스크립트별로 나눠져있던 temp 데이터 합치기
for folder in sess_FList:
  lst = []
  sess_save=[] 
  temp_FList = os.listdir(path_dir+'/'+folder)
  for script in temp_FList:
      with open('./dataset/KEMDy20_v1_1/TEMP/'+folder+'/'+script) as lines:
        b = list(map(str, script.split('_')))
        for line in lines:
          data = line.strip().split(',')
          if len(data)==3:
            a = list(map(str, data[2].split('_')))
            data[2] = data[2].replace(a[0],b[0])
            lst.append(data)
  df = pd.DataFrame(lst, columns=['data','date','sess'])
  df = df.astype({'data':float})  
  
  sess_name = df.sess.unique()
  for sess in tqdm(sess_name):
    eda=list(df[df.sess == sess].data.values)
    sess_list = [sess, eda]
    sess_save.append(sess_list)

  sess_save_df = pd.DataFrame(sess_save, columns=['session', 'temp'])
  sess_save_df.to_pickle(drop_dir+folder+'_temp.pkl')

전체 바이오 데이터 합치기

In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import os

# 세션별 저장한 바이오(eda, temo)파일 경로 지정
eda_dir = './dataset/KEMDy20_v1_1/sensor/EDA'
temp_dir = './dataset/KEMDy20_v1_1/sensor/Temp'

# 지젇된 경로 파일 불러오기 및 정렬렬
eda_sess_FList = sorted(os.listdir(eda_dir))
temp_sess_FList = sorted(os.listdir(temp_dir))

# 빈 데이터 프레임 만들기
bio_all = pd.DataFrame(columns=['segment_id','eda','temp'])

# 세션별 저장된 바이오(eda,temp) 데이터 합치기
for eda, temp in zip(eda_sess_FList, temp_sess_FList):
  print(eda, temp)
  TEMP = pd.read_pickle('./dataset/KEMDy20_v1_1/sensor/Temp/'+ temp )
  EDA = pd.read_pickle('./dataset//KEMDy20_v1_1/sensor/EDA/'+ eda )
  TEMP.columns = ['segment_id','temp']
  EDA.columns = ['segment_id','eda']
  bio = EDA.merge(TEMP, on ='segment_id')
  bio_all = pd.concat([bio_all,bio] ignore_index=True)

  bio_all

segment_id를 기준으로 annotation데이터와 합치기

In [ ]:
# 전처리된 annotation 데이터 불러오기

train = pd.read_pickle('./dataset/KEMDy20_v1_1/new/annotation/train_origin.pkl')
test = pd.read_pickle('./dataset/KEMDy20_v1_1/new/annotation/test_origin.pkl')

In [ ]:
# annotation 데이터와 segment_id 기준으로 merge

bio_train = train.merge(bio_all, how = 'inner', on = 'segment_id')
bio_test = test.merge(bio_all, how = 'inner', on = 'segment_id')

In [ ]:
# 최종종 바이오 데이터 프레임 저장

bio_train.to_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/train_origin.pkl')
bio_test.to_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/test_origin.pkl')

#데이터 증강 및 패딩 과정

In [ ]:
# 데이터 불러오기
train = pd.read_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/train_origin.pkl') 
test = pd.read_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/test_origin.pkl')

In [ ]:
# 패딩 함수
def get_numpy_from_nonfixed_2d_array(aa, fixed_length=141):
    rows = []
    for a in aa:
        rows.append(np.pad(a, (0, fixed_length), 'constant', constant_values=0)[:fixed_length])
    return np.concatenate(rows, axis=0).reshape(-1, fixed_length)

In [ ]:
# 데이터 패딩 진행
temp = get_numpy_from_nonfixed_2d_array(train['temp'])
eda = get_numpy_from_nonfixed_2d_array(train['eda'])
train_y = train.emotion_id.values

"기쁨" 맞춰 데이터 증강

In [ ]:
Temp=pd.DataFrame(temp)
Temp['emotion']=pd.DataFrame(train_y)

EDA = pd.DataFrame(eda)
EDA['emotion']=pd.DataFrame(train_y)

nedf = EDA.loc[(EDA.emotion==4),:].drop(['emotion'],axis=1).to_numpy()
ntdf = Temp.loc[(Temp.emotion==4),:].drop(['emotion'],axis=1).to_numpy()
n_y = Temp.loc[(Temp.emotion==4),'emotion'].to_numpy()

edf = EDA.loc[(EDA.emotion!=4),:].drop(['emotion'],axis=1).to_numpy()
tdf = Temp.loc[(Temp.emotion!=4),:].drop(['emotion'],axis=1).to_numpy()
y =EDA.loc[(EDA.emotion!=4),'emotion'].to_numpy()

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state= 1024)
temp_x,t_y = smote.fit_resample(tdf,y)
eda_x,t_y = smote.fit_resample(edf,y)

In [ ]:
htemp = np.concatenate((temp_x,ntdf), axis=0)
heda = np.concatenate((eda_x,nedf), axis=0)
h_y = np.concatenate((t_y,n_y), axis=0)

In [ ]:
import numpy as  np
import pandas as pd
import pickle
import torch

# annotation과 합친 바이오데이터 파일 불러오기기
train = pd.read_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/train_origin.pkl')
test = pd.read_pickle('./dataset/KEMDy20_v1_1/sensor/bio_train/test_origin.pkl')

# 데이터 padding
temp = get_numpy_from_nonfixed_2d_array(train['temp'])
eda = get_numpy_from_nonfixed_2d_array(train['eda'])
train_y = train.emotion_id.values

# padding한 array에서 "중립" 클래스 데이터 제외
Temp=pd.DataFrame(temp)
Temp['emotion']=pd.DataFrame(train_y)

EDA = pd.DataFrame(eda)
EDA['emotion']=pd.DataFrame(train_y)

nedf = EDA.loc[(EDA.emotion==4),:].drop(['emotion'],axis=1).to_numpy()
ntdf = Temp.loc[(Temp.emotion==4),:].drop(['emotion'],axis=1).to_numpy()
n_y = Temp.loc[(Temp.emotion==4),'emotion'].to_numpy()

edf = EDA.loc[(EDA.emotion!=4),:].drop(['emotion'],axis=1).to_numpy()
tdf = Temp.loc[(Temp.emotion!=4),:].drop(['emotion'],axis=1).to_numpy()
y =EDA.loc[(EDA.emotion!=4),'emotion'].to_numpy()

In [ ]:
# 데이터 증강
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state= 1024)
temp_x,t_y = smote.fit_resample(tdf,y)
eda_x,t_y = smote.fit_resample(edf,y)

In [ ]:
# "중립" 클래스 데이터와 합치기기
htemp = np.concatenate((temp_x,ntdf), axis=0)
heda = np.concatenate((eda_x,nedf), axis=0)
h_y = np.concatenate((t_y,n_y), axis=0)

In [ ]:
# 증강한 바이오 데이터 배열 저장
np.save('./dataset/KEMDy20_v1_1/sensor/bio_train/temp_x.npy',htemp)
np.save('./dataset/KEMDy20_v1_1/sensor/bio_train/eda_x.npy',heda)
np.save('./dataset/KEMDy20_v1_1/sensor/bio_train/t_y.npy',h_y)